<a href="https://colab.research.google.com/github/Kim-TaeKyoung/ai_championship/blob/main/Only_YOLO_Inference_and_Export.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 기본 필요 패키지
import os
import sys
import random
import json
import time
import datetime
import yaml
import glob
import ast

import numpy as np
from tqdm.notebook import tqdm
import torchvision
import pandas as pd
import torch

from IPython.display import Image, clear_output, display
from IPython.core.magic import register_line_cell_magic

!pip install natsort

# 구글 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

# 패키지 설치 (EfficientDet)
!cd /content/drive/MyDrive/Colab/AI_ChampionShip/EfficientDet && cat requirements_colab.txt | xargs -n 1 -L 1 pip install
#!pip install --force https://github.com/chengs/tqdm/archive/colab.zip
!pip install efficientnet_pytorch
!pip install tensorboardX
sys.path.append("/content/drive/MyDrive/Colab/AI_ChampionShip/EfficientDet/");

# EfficientDet 임포트
from train_detector import Detector as EDDetector
from infer_detector import Infer as EDInfer

# YOLOv5 설치
!cd /content/drive/MyDrive/Colab/AI_ChampionShip/YOLOv5 && pip install -qr requirements.txt
sys.path.append("/content/drive/MyDrive/Colab/AI_ChampionShip/YOLOv5/");
from utils.plots import plot_results


# EfficientNet 임포트
!pip install efficientnet_pytorch

from efficientnet_pytorch import EfficientNet

# Weighted Box Fusion 설치 및 임포트
!pip install ensemble-boxes
from ensemble_boxes import weighted_boxes_fusion as WBF

In [ ]:
# 각 모델의 weights
# ED_weight_dir = "/content/drive/MyDrive/Colab/AI_ChampionShip/EfficientDet/weight/2021-09-30_17_02_38"
YOLO_weight_dir = '/content/drive/MyDrive/Colab/AI_ChampionShip/YOLOv5/runs/train/20211029-170019_7.8k/weights/best.pt'
EN_weight_dir = '/content/drive/MyDrive/Colab/AI_ChampionShip/EfficientNet/weights_2021-10-19-01:39:28_best_229'

# EfficientNet에서 사용할 입력 프레임들
test_data_dir = '/content/drive/MyDrive/Colab/AI_ChampionShip/dataset/test-frame_rearranged/*'

# 화재로 판단된 결과들의 이미지 저장 장소
img_output_dir = '/content/drive/MyDrive/Colab/AI_ChampionShip/results/'

# Confidence
YOLO_conf = 0.1
# ED_conf = 0.4

# result_mode일 경우 화재로 판단한 프레임에서 이미지를 제외한 결과를 출력하며, 아닐 경우 판단하는 모든 프레임의 이미지를 포함한 결과를 출력함
result_mode = True

# Box 가중치 (순서대로 YOLO, EfficientDet)
NMS_weights = [2]
# NMS IOU threshold
NMS_iou_thr = 0.5
# 무시할 confidence threshold
NMS_skip_box_thr = 0.9

In [ ]:
%matplotlib inline

import os
import warnings
import shutil

from PIL import Image as PILImage
from matplotlib import pyplot as plt
from IPython.display import Image, clear_output, display
from IPython.core.magic import register_line_cell_magic
from natsort import natsorted

def get_current_segment(string):
  '''
  현재 이미지의 파일이름에서 영상 ID를 반환함
  string - 파일 이름
  '''
  return os.path.basename(string).split(']')[0].replace('[','')

def return_torch_tensor(img, width, height):
  '''
  이미지의 넓이 높이를 주어진 값으로 바꾸고, unsqueeze하여 배치 사이즈가 포함된 torch Tensor를 반환함
  EfficientNet Input에 필요함

  img - 이미지 배열, width - 넓이, height - 높이
  '''
  X = img.resize((width, height))
  X = np.array(X)
  X = np.transpose(X, [2,1,0])
  X = np.reshape(X, [1, *X.shape])
  return torch.tensor(X, dtype=torch.float32)

def normalize(val, min, max):
  '''
  값을 최대 최소를 받아 [0,1]의 범위로 정규화함
  여기서 사용한 Weighted Box Fusion은 정규화된 데이터가 필요로함

  일부 값에서 실제 이미지크기보다 크게 반환되는
  (정규화 값이 1보다 크게 나오거나 0보다 작게 나오는)
  값들은 각각 1과 0으로 floor, ceil함

  val - 대상 값, min - 최소, max - 최대
  '''
  normalized = (val - min) / max
  if normalized > 1:
    normalized = 1
  elif normalized < 0:
    normalized = 0
  return normalized

def unnormalize(val, min, max):
  '''
  정규화한 값을 되돌리는 함수

  val - 대상 값, min - 최소, max - 최대
  '''
  return (val + min) * max

# Dictionary 형태로 반환된 값을 받아 각각 bbox, score, label들을 담은 리스트로 반환함 (bbox의 좌표는 정규화 함)
def bbox_packer(results, width, height):
  '''
  Dictionary 형태로 주어진 값을 받아 각각 bbox, score, label들을 담은 리스트로 반환함
  여기서 bbox의 값들은 Weight Fusion Box를 위해 정규화 됨

  results = 결과당 xmin, ymin, xmax, ymax, confidence, class를 담은 Dictionary
  width = 이미지의 넓이
  height = 이미지의 높이
  '''
  tmp_boxes = []
  tmp_scores = []
  tmp_labels = []

  for i in results:
    tmp_boxes.append([
          normalize(i['xmin'], 0, width),
          normalize(i['ymin'], 0, height),
          normalize(i['xmax'], 0, width),
          normalize(i['ymax'], 0, height)])
    tmp_scores.append(i['confidence'])
    tmp_labels.append(i['class'])
  return tmp_boxes, tmp_scores, tmp_labels

# YOLOv5 Inference 함수
def yolo(model, img, conf, save_result: bool):
  '''
  YOLOv5 Inference 함수
  
  model - YOLOv5 모델, img - 이미지 경로 또는 이미지 배열
  conf - confidence threshold, save_results - bbox를 적용한 이미지를 저장 할 것인지
  '''
  yoloy = None
  result = None
  model.conf = conf

  result = model(img)
  yoloy = ast.literal_eval(result.pandas().xyxy[0].to_json(orient="records"))
  if save_result:
    result.save(save_dir='./')

  return yoloy

# def effdet(model, img_path: str, class_list: list, conf):
#   '''
#   EfficientDet Inference 함수

#   model - EfficientDet 모델
#   img_path - 이미지 경로
#   class_list - 결과 레이블
#   conf - confidence threshold
#   '''
#   scores = []
#   labels = []
#   boxes = []

#   effdety = []
#   rtn = None

#   rtn = model.Predict(img_path, class_list, vis_threshold=conf)

#   if rtn:
#     _, scores, labels, boxes = rtn

#     for i in zip(scores, labels, boxes):
#       if i[0].item() > conf:
#         effdety.append({'xmin' : i[2][0].item(), 'ymin' : i[2][1].item(), 'xmax' : i[2][2].item(), 'ymax' : i[2][3].item(), \
#                         'confidence' : i[0].item(), 'class' : i[1].item()})
#   return effdety

def effnet(model, img, width: int, height: int):
  '''
  EfficientNet Inference 함수
  
  model - EfficientNet 모델
  img - torch Tensor 이미지 배열
  width - 모델 입력에 필요한 이미지 넓이
  height - 모델 입력에 필요한 이미지 높이
  conf - confidence threshold
  '''
  effnety = None

  X = return_torch_tensor(img, width, height)

  with torch.no_grad():
      X = X.to(device)
      yhat = model(X)
      yhat = torch.argmax(yhat, -1)

  effnety = 0 if yhat == 1 else 1
  return effnety

In [ ]:
import cv2

def drawBoundingBoxes(img_path, img_result_path, infres, colors):
  '''
  bbox를 적용한 이미지를 반환함
  Weight Box Fusion의 결과 이미지를 생성하는데 사용됨

  img_path - 원본 이미지
  img_result_path - 결과 이미지 저장 경로
  infres - box의 위치 정보 (xmin,ymin,xmax,ymax)와 score 그리고 label이 포함된 리스트들의 리스트
  colors - 각 label에서 사용할 색깔들 (tuple로 (b,g,r))
  '''
  img = cv2.imread(img_path)
  for res in infres:
    box, score, label_t = res

    left = int(box[0])
    top = int(box[1])
    right = int(box[2])
    bottom = int(box[3])

    label = 'fire' if label_t == 0 else 'smoke'
    color = colors[0] if label_t == 0 else colors[1]

    height, width, _ = img.shape
    thick = int((height + width) // 900)

    cv2.rectangle(img, (left, top), (right, bottom), color, thick)
    cv2.putText(img, label + ' ' + score.astype('str'), (left, top - 12), 0, 1e-3 * height, color, thick//3)

  cv2.imwrite(img_result_path, img)

In [ ]:
# Preview 지속 출력을 위한 Pyplot figure 제한해제
plt.rcParams.update({'figure.max_open_warning': 0})

# 프레임으로 잘라진 이미지를 불러와서 ID 순서대로 정렬함
test_images = natsorted(glob.glob(test_data_dir))
# 영상 ID별로 저장될 결과 Dictionary를 indexing 하기 위해 이미지에서 unique한 영상 ID를 추출함 
unique_test_images = set([get_current_segment(i) for i in test_images])

'''
YOLO와 EfficientDet은 자체적으로 모델을 CUDA에 로드하고, 이미지를 모델에 맞게 편집함
'''

# YOLO Model definition
yolo_model = torch.hub.load('ultralytics/yolov5', 'custom', path=YOLO_weight_dir, verbose=False)

# EfficientDet Model definition
# effdet_model = EDInfer()
# effdet_model.Model(model_dir=ED_weight_dir)

# - Class list for EfficientDet Model
# effdet_class_list = ['fire', 'smoke']

# EfficientNet Model definition
device = torch.device('cuda')
effnet_model = EfficientNet.from_name('efficientnet-b1', num_classes=2).eval()
effnet_model.load_state_dict(torch.load(EN_weight_dir))
effnet_model.to(device)

# Prepare dictionary for saving prediction
# Each has a number of sorted test_image index as key

'''
Inference 결과들을 저장할 Dictionary
키의 값은 영상 ID를 의미함.

각 영상의 프레임 파일이름에 있는 영상 ID를 Index로 사용하며,
순차적으로 정렬된 프레임들을 각각 판단하고 해당 프레임의 영상 ID의 key에 결과를 저장함
'''

results = {}

for i in range(len(unique_test_images)):
  results.update({str(i) : []})

# EfficientDet 결과 일부가 실제 이미지 크기보다 크게 나오는 경우가 있는데 이를 catch하기 위함
warnings.simplefilter("error", UserWarning)

for img_path in tqdm(iterable=test_images):
  current_segment = get_current_segment(img_path)
  img = PILImage.open(img_path)
  
  # ========== MODEL INFERENCE CODE GOES HERE! ==========

  # YOLO (Image Processing is built-in)
  yoloy = yolo(yolo_model, img, YOLO_conf, save_result=True)

  # EffDet (Image Processing is built-in)
  # effdety = effdet(effdet_model, img_path, effdet_class_list, ED_conf)

  # EffNet
  effnety = effnet(effnet_model, img, 224, 224)

  if not result_mode:
    print('YOLO : ', yoloy)
    # print('EfficientDet : ', effdety)
    print('EfficientNet : ', effnety)

  # ========== MODEL INFERENCE CODE ENDS HERE! ==========

  # ========== NMS & ENSAMBLE, Decision Strategy ==========

  # (1). YOLO + EffDet
  '''
  YOLO와 EfficientDet의 결과를 받아, 두 모델 모두 반환 결과가 있을 경우에만 (불과 연기를 판단한 경우)
  두 결과를 합쳐 Weighted Box Fusion을 적용함.

  EfficientNet의 판단(화재인가 아닌가)이 참일 경우 최종적으로 해당 프레임을 화재로 판단
  '''

  boxes = []
  scores = []
  labels = []

  if yoloy:
    width, height = img.size
    for i in [yoloy]:
      b, s, l = bbox_packer(i, width, height)
      boxes.append(b)
      scores.append(s)
      labels.append(l)

    try:
      boxes, scores, labels = WBF(boxes, \
                            scores, \
                            labels, \
                            weights=NMS_weights, \
                            iou_thr=NMS_iou_thr, \
                            skip_box_thr=NMS_skip_box_thr)
      
      for bs in boxes:
        for i, b in enumerate(bs):
          # xmin = 0, ymax = 1, xmax = 2, ymax = 3
          if i % 2 == 0:
            bs[i] = unnormalize(bs[i], 0, width)
          else:
            bs[i] = unnormalize(bs[i], 0, height)

    except UserWarning as e:
      print('Big box occurred')
      print(boxes, scores, labels)

  # Object Detection 모델들의 결과 이미지의 위치들
  yolo_output_img = os.path.basename(img_path)
  # effdet_output_img = os.path.basename('output.jpg')
  # Weighted Box Fusion의 결과를 저장할 위치
  nms_output_img = img_output_dir + str(current_segment) + '/' + 'nms' + os.path.basename(img_path)

  if len(boxes) and effnety == 1:
    Y = 1
    if result_mode:
      print('Fire Found, ', img_path)
    
    if not os.path.exists(img_output_dir + str(current_segment)):
      os.mkdir(img_output_dir + str(current_segment))

    # Weighted Box Fusion의 결과를 만듦
    drawBoundingBoxes(img_path, \
                      nms_output_img, \
                      zip(boxes, scores, labels), \
                      [(0, 255, 0), (0, 0, 255)])
  
    result_img_paths = [yolo_output_img, nms_output_img]
    result_img_modifiers = ['yolo', 'nms']

    if not result_mode:
      fig = plt.figure(figsize=(30,30))
      columns = 2
      rows = 1

      for i, data in enumerate(zip(result_img_paths, result_img_modifiers)):
        img = PILImage.open(data[0])
        sub = fig.add_subplot(1, 2, i + 1)
        sub.text(100, 100, data[1] , style='italic', bbox={'facecolor': 'grey', 'alpha': 0.5, 'pad': 10})
        plt.imshow(img)
      plt.show()

    # 결과 이미지를 저장할 위치로 옮김
    shutil.move(yolo_output_img, img_output_dir + str(current_segment) + '/' + result_img_modifiers[0] + yolo_output_img)
    # shutil.move(effdet_output_img, img_output_dir + str(current_segment) + '/' + result_img_modifiers[1] + yolo_output_img)

  else:
    Y = 0
    os.remove(yolo_output_img)

  # =======================================

  results[str(current_segment)].append(Y)
  if not result_mode:
    print(Y)

In [ ]:
# Replace list indices to corresponding filenames
with open('/content/drive/MyDrive/Colab/AI_ChampionShip/dataset/raw/kdx-dataset/test/sample_fire_scenes.json', 'r') as f:
  submit = json.load(f)

for i, s in enumerate(submit.keys()):
  results.update({s : results[str(i)]})
  del results[str(i)]

In [ ]:
print(results)
print(len(results))

{'20130113220042_20_552_1039939_360.mp4': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], '20140117211758_20_552_1062194_360.mp4': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], '20150328205830_20_552_1093116_360.mp4': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [ ]:
# Post-processing
# State machine for submission format

converted_dict = {}

def start_new_state(curr_list, i):
  curr_list.append(i)

def end_current_state(curr_list, full_list, i):
  curr_list.append(i)
  full_list.append(curr_list)

def end_state(curr_list, full_list):
  full_list.append(curr_list)

for strings, predictions in results.items():
  print(strings, predictions)
  
  previous_state = 0
  current_list = []
  replacement_list = []
  
  for i, p in enumerate(predictions):
    current_state = p

    if previous_state == 0:
      if current_state == 0:
        # Ignore non-fire state
        continue
      elif current_state == 1:
        start_new_state(current_list, i)
        previous_state = current_state
    
    if previous_state == 1:
      if current_state == 0:
          end_current_state(current_list, replacement_list, i)
          previous_state = current_state
          current_list = []
      elif current_state == 1:
        # Ignore fire state
        continue

    if i + 1 == len(predictions):
      if previous_state == 0:
        continue
      elif previous_state == 1:
        end_state(current_list, replacement_list)
        

    converted_dict.update({strings : replacement_list})
    len(converted_dict)

In [ ]:
print(json.dumps(converted_dict, sort_keys=False, indent=2))
print(results)

In [ ]:
for key in converted_dict:
  submit[key]['scenes'] = converted_dict[key]

print(json.dumps(submit, sort_keys=False, indent=2))

with open('/content/drive/MyDrive/Colab/AI_ChampionShip/submission/inference-' + datetime.datetime.now().strftime('%Y%m%d-%H%M%S') + '-' + str(YOLO_conf) + '-'+'.json', 'w') as f:
  json.dump(submit, f, indent=2)